In [1]:
!sudo apt-get install -y pciutils
!curl https://ollama.ai/install.sh | sh
!pip install ultimate-sitemap-parser langchain-community tiktoken langchain-openai langchainhub langchain langgraph langchain-text-splitters langchain-pinecone bs4 nest_asyncio tavily-python

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids
The following NEW packages will be installed:
  libpci3 pci.ids pciutils
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 343 kB of archives.
After this operation, 1,581 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 pci.ids all 0.0~2022.01.22-1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Fetched 343 kB in 1s (520 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: unable to initializ

**Setting up code to run ollama in the background**

In [2]:
import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

**Start Ollama Service**

In [3]:
ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

**Pulling LLAMA3 as main LLM and nomic-embed-text for the embedding**

In [4]:
!ollama pull llama3
!ollama pull nomic-embed-text

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕

**Checking the model list**

In [5]:
!ollama list

NAME                   	ID          	SIZE  	MODIFIED               
nomic-embed-text:latest	0a109f422b47	274 MB	Less than a second ago	
llama3:latest          	365c0bd3c000	4.7 GB	13 seconds ago        	


**Begin the main section of the program:**

In [6]:
import os
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import OllamaEmbeddings
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain.schema import Document
from langchain_community.tools.tavily_search import TavilySearchResults
os.environ['PINECONE_API_KEY'] = '66d9b856-aeba-405a-8be2-d0d4bf9f4cb3'
os.environ['TAVILY_API_KEY']= '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'

**Building out the functions as nodes:**

In [7]:
def fetch_retriever():
    embeddings= OllamaEmbeddings(model="nomic-embed-text")
    vectorstore = PineconeVectorStore(index_name="snowflake-docs-rag", embedding=embeddings)
    return vectorstore.as_retriever(search_kwargs={"k":3})



def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


### Retriever Node

def retrieve(state):
    print("---RETRIEVING DOCUMENTS---")
    state_dict = state["keys"]
    question = state_dict["question"]
    retriever=fetch_retriever()
    documents = retriever.invoke(question)
    return {"keys": {"documents": documents, "question": question}}

###Grader Node

def relevance_grader(state):

    print("---CHECK RELEVANCE---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]
    llm = ChatOllama(model="llama3", format="json", temperature=0)
    prompt = PromptTemplate(
        template="""You are a grader assessing relevance of a retrieved document to a user question. \n
        Here is the retrieved document: \n\n {context} \n\n
        Here is the user question: {question} \n
        If the document contains keywords related to the user question, grade it as relevant. \n
        It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
        Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
        Provide the binary score as a JSON with a single key 'score' and no premable or explaination.""",
        input_variables=["question","context"],
    )

    chain = prompt | llm | JsonOutputParser()

    # Score
    filtered_docs = []
    search = "No"  # Default do not opt for web search to supplement retrieval
    for d in documents:
        score = chain.invoke(
            {
                "question": question,
                "context": d.page_content,
            }
        )
        grade = score["score"]
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            search = "Yes"  # Perform web search
            continue

    return {
        "keys": {
            "documents": filtered_docs,
            "question": question,
            "run_web_search": search,
        }
    }

    return retrieval_grader

##Generator Node
def generate(state):
    prompt = PromptTemplate(
        template="""Imagine you are an expert in snowflake platform and you are to asnwer the user query with this context: \n\n {context} \n\n
        Here is the user question: {question} \n
        """,
        input_variables=["context","question"],
    )

    llm = ChatOllama(model="llama3", temperature=0)
    rag_chain = prompt | llm | StrOutputParser()

    print("---GENERATE---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]


    # Chain
    rag_chain = prompt | llm | StrOutputParser()

    # Run
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {
        "keys": {"documents": documents, "question": question, "generation": generation}
    }



#Rewriter Node
def question_rewrite(state):
    llm = ChatOllama(model="llama3", temperature=0)
    print("---TRANSFORM QUERY---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]


    # Create a prompt template with format instructions and the query
    prompt = PromptTemplate(
        template="""You are generating questions that is well optimized for retrieval in the context of snowflake platform. \n
        Look at the input and try to reason about the underlying sematic intent / meaning. \n
        Here is the initial question:
        \n ------- \n
        {question}
        \n ------- \n
        Answer with just the new formulated question: """ ,
        input_variables=["question"],
    )
     # Prompt
    chain = prompt | llm | StrOutputParser()
    better_question = chain.invoke({"question": question})

    return {
        "keys": {"documents": documents, "question": better_question}
    }


##Web search node:


def web_search(state):

    print("---WEB SEARCH---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]

    tool = TavilySearchResults()
    docs = tool.invoke({"query": question})

    for idx, d in enumerate(docs):
      if not isinstance(d, dict):
        raise TypeError(f"Element at index {idx} is not a dictionary: {docs}")
    filtered_contents = [d["content"] for d in docs if d["content"] is not None]
    web_results = "\n".join(filtered_contents)
    web_results = Document(page_content=web_results)
    documents.append(web_results)

    return {"keys": {"documents": documents, "question": question}}









**Building an edge to seperate the flows **

In [8]:
### Edges


def decide_to_generate(state):
    """
    Determines whether to generate an answer or re-generate a question for web search.

    Args:
        state (dict): The current state of the agent, including all keys.

    Returns:
        str: Next node to call
    """

    print("---DECIDE TO GENERATE---")
    state_dict = state["keys"]
    question = state_dict["question"]
    filtered_documents = state_dict["documents"]
    search = state_dict["run_web_search"]

    if search == "Yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print("---DECISION: TRANSFORM QUERY and RUN WEB SEARCH---")
        return "question_rewrite"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"

**Define the GraphState Class**

In [9]:
from typing import List

from typing_extensions import Dict, TypedDict


class GraphState(TypedDict):
    """
    Attributes:
        question: question
        generation: LLM generation
        web_search: whether to add search
        documents: list of documents
    """

    keys: Dict[str, any]

**Building the graph**

In [10]:
import pprint
from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("relevance_grader", relevance_grader)  # grade documents
workflow.add_node("generate", generate)  # generatae

workflow.add_node("question_rewrite", question_rewrite)  # transform_query
workflow.add_node("web_search", web_search)  # web search

# Build graph
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "relevance_grader")
workflow.add_conditional_edges(
    "relevance_grader",
    decide_to_generate,
    {
        "question_rewrite": "question_rewrite",
        "generate": "generate",
    },
)
workflow.add_edge("question_rewrite", "web_search")
workflow.add_edge("web_search", "generate")
workflow.add_edge("generate", END)

# Compile
app = workflow.compile()

Input  Execution

In [11]:
# Run
inputs = {
    "keys": {
        "question": "For loop in snowflake"
    }
}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint.pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint.pprint("\n---\n")

# Final generation
pprint.pprint(value['keys']['generation'])

---RETRIEVING DOCUMENTS---
"Node 'retrieve':"
'\n---\n'
---CHECK RELEVANCE---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---DECIDE TO GENERATE---
---DECISION: TRANSFORM QUERY and RUN WEB SEARCH---
"Node 'relevance_grader':"
'\n---\n'
---TRANSFORM QUERY---
"Node 'question_rewrite':"
'\n---\n'
---WEB SEARCH---
"Node 'web_search':"
'\n---\n'
---GENERATE---
"Node 'generate':"
'\n---\n'
("I'd be happy to help!\n"
 '\n'
 'In Snowflake, you can use the `FOR` loop or the `CONTINUE` (or `ITERATE`) '
 'command to perform iterative operations. Here are some syntax and usage '
 'guidelines for each:\n'
 '\n'
 '**FOR Loop**\n'
 '\n'
 'A `FOR` loop repeats a sequence of steps a specific number of times. The '
 'number of times might be specified by the user, or might be specified by the '
 'number of rows in a cursor.\n'
 '\n'
 'Syntax:\n'
 '```sql\n'
 'FOR variable IN (SELECT ... FROM ...) DO\n'
 '  -- statements to execute for each iter

**Streamlit Integration**


In [12]:
! pip install streamlit -q
!wget -q -O - ipv4.icanhazip.com

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.0 MB/s eta 0:00:00
34.34.108.224


In [13]:
%%writefile app.py
import os
import threading
import subprocess
import requests
import json
import streamlit as st
def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()
# Set the title of the app
st.title('Snowflake Sensei')
import os
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import OllamaEmbeddings
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain.schema import Document
from langchain_community.tools.tavily_search import TavilySearchResults
os.environ['PINECONE_API_KEY'] = '66d9b856-aeba-405a-8be2-d0d4bf9f4cb3'
os.environ['TAVILY_API_KEY']= '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'

def fetch_retriever():
    embeddings= OllamaEmbeddings(model="nomic-embed-text")
    vectorstore = PineconeVectorStore(index_name="snowflake-docs-rag", embedding=embeddings)
    return vectorstore.as_retriever(search_kwargs={"k":3})



def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


### Retriever Node

def retrieve(state):
    print("---RETRIEVING DOCUMENTS---")
    state_dict = state["keys"]
    question = state_dict["question"]
    retriever=fetch_retriever()
    documents = retriever.invoke(question)
    return {"keys": {"documents": documents, "question": question}}

###Grader Node

def relevance_grader(state):

    print("---CHECK RELEVANCE---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]
    llm = ChatOllama(model="llama3", format="json", temperature=0)
    prompt = PromptTemplate(
        template="""You are a grader assessing relevance of a retrieved document to a user question. \n
        Here is the retrieved document: \n\n {context} \n\n
        Here is the user question: {question} \n
        If the document contains keywords related to the user question, grade it as relevant. \n
        It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
        Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
        Provide the binary score as a JSON with a single key 'score' and no premable or explaination.""",
        input_variables=["question","context"],
    )

    chain = prompt | llm | JsonOutputParser()

    # Score
    filtered_docs = []
    search = "No"  # Default do not opt for web search to supplement retrieval
    for d in documents:
        score = chain.invoke(
            {
                "question": question,
                "context": d.page_content,
            }
        )
        grade = score["score"]
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            search = "Yes"  # Perform web search
            continue

    return {
        "keys": {
            "documents": filtered_docs,
            "question": question,
            "run_web_search": search,
        }
    }

    return retrieval_grader

##Generator Node
def generate(state):
    prompt = PromptTemplate(
        template="""Imagine you are an expert in snowflake platform and you are to asnwer the user query with this context: \n\n {context} \n\n
        Here is the user question: {question} \n
        """,
        input_variables=["context","question"],
    )

    llm = ChatOllama(model="llama3", temperature=0)
    rag_chain = prompt | llm | StrOutputParser()

    print("---GENERATE---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]


    # Chain
    rag_chain = prompt | llm | StrOutputParser()

    # Run
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {
        "keys": {"documents": documents, "question": question, "generation": generation}
    }



#Rewriter Node
def question_rewrite(state):
    llm = ChatOllama(model="llama3", temperature=0)
    print("---TRANSFORM QUERY---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]


    # Create a prompt template with format instructions and the query
    prompt = PromptTemplate(
        template="""You are generating questions that is well optimized for retrieval in the context of snowflake platform. \n
        Look at the input and try to reason about the underlying sematic intent / meaning. \n
        Here is the initial question:
        \n ------- \n
        {question}
        \n ------- \n
        Answer with just the new formulated question: """ ,
        input_variables=["question"],
    )
     # Prompt
    chain = prompt | llm | StrOutputParser()
    better_question = chain.invoke({"question": question})

    return {
        "keys": {"documents": documents, "question": better_question}
    }


##Web search node:


def web_search(state):

    print("---WEB SEARCH---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]

    tool = TavilySearchResults()
    docs = tool.invoke({"query": question})

    for idx, d in enumerate(docs):
      if not isinstance(d, dict):
        raise TypeError(f"Element at index {idx} is not a dictionary: {docs}")
    filtered_contents = [d["content"] for d in docs if d["content"] is not None]
    web_results = "\n".join(filtered_contents)
    web_results = Document(page_content=web_results)
    documents.append(web_results)

    return {"keys": {"documents": documents, "question": question}}


### Edges


def decide_to_generate(state):
    """
    Determines whether to generate an answer or re-generate a question for web search.

    Args:
        state (dict): The current state of the agent, including all keys.

    Returns:
        str: Next node to call
    """

    print("---DECIDE TO GENERATE---")
    state_dict = state["keys"]
    question = state_dict["question"]
    filtered_documents = state_dict["documents"]
    search = state_dict["run_web_search"]

    if search == "Yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print("---DECISION: TRANSFORM QUERY and RUN WEB SEARCH---")
        return "question_rewrite"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"

from typing import List

from typing_extensions import Dict, TypedDict


class GraphState(TypedDict):
    """
    Attributes:
        question: question
        generation: LLM generation
        web_search: whether to add search
        documents: list of documents
    """

    keys: Dict[str, any]


import pprint
from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("relevance_grader", relevance_grader)  # grade documents
workflow.add_node("generate", generate)  # generatae

workflow.add_node("question_rewrite", question_rewrite)  # transform_query
workflow.add_node("web_search", web_search)  # web search

# Build graph
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "relevance_grader")
workflow.add_conditional_edges(
    "relevance_grader",
    decide_to_generate,
    {
        "question_rewrite": "question_rewrite",
        "generate": "generate",
    },
)
workflow.add_edge("question_rewrite", "web_search")
workflow.add_edge("web_search", "generate")
workflow.add_edge("generate", END)

# Compile
app = workflow.compile()


# Take user input
user_query = st.text_input('Enter you snowflake related query:')

# Respond with a greeting when the user provides their name
if user_query:
    # Run
    inputs = {
    "keys": {
        "question": user_query
    }
    }
    for output in app.stream(inputs):
     for key, value in output.items():
        # Node
        pprint.pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
     pprint.pprint("\n---\n")

    # Final generation
    result=value['keys']['generation']
    st.write(f'{result}')


Writing app.py


In [14]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.34.108.224:8501

npx: installed 22 in 7.399s
your url is: https://loud-badgers-lay.loca.lt
Error: listen tcp 0.0.0.0:11434: bind: address already in use
Error: listen tcp 0.0.0.0:11434: bind: address already in use
---RETRIEVING DOCUMENTS---
"Node 'retrieve':"
'\n---\n'
---CHECK RELEVANCE---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---DECIDE TO GENERATE---
---DECISION: TRANSFORM QUERY and RUN WEB SEARCH---
"Node 'relevance_grader':"
'\n---\n'
---TRANSFORM QUERY---
"Node 'question_rewrite':"
'\n---\n'
---WEB SEARCH---
"Node 'web_search':"
'\n---\n'
---GENERATE---
"Node 'generate':"
'\n---\n'
  Stopping...
^C
